## **Homework 3**
### Engineering Image Analysis - Combustion Images
#### Step 1 - Train 2D CNN for Classifying Flames

In [2]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from matplotlib.pyplot import imshow
%matplotlib inline

#Load the data from the .mat file 
#NOTE: x data are single channel images sized at 250x100 pixels 
#      y data are classifiers for the x images stating stable (1) and unstable (0)

with h5py.File('combustion_img_13.mat', 'r') as f:
    print(list(f.keys())) #Print the available keys in the file
    train_set_x = np.array(f['train_set_x'])
    train_set_y = np.array(f['train_set_y'])
    valid_set_x = np.array(f['valid_set_x'])
    valid_set_y = np.array(f['valid_set_y'])
    test_set_x = np.array(f['test_set_x'])
    test_set_y = np.array(f['test_set_y'])    

print(train_set_x.shape)
print(train_set_y.shape)

#By analyzing data in matlab, found a split point between y data 0 & 1 values to determine which value corresponds to stable/unstable
imgs = []
for i in range(5995, 6005):           
    sample_img = train_set_x[:, i]
    #print(f"initial shape: {sample_img.shape}")
    sample_img = sample_img.reshape(250, 100)
    #print(f"next shape: {sample_img.shape}")
    imgs.append(sample_img)
    
stacked = np.hstack(imgs)
# imshow(stacked, cmap='hot')


['test_set_x', 'test_set_y', 'train_set_x', 'train_set_y', 'valid_set_x', 'valid_set_y']
(25000, 54000)
(54000, 1)


In [3]:
# Preprocess the data via normalization and converting the labels to one-hot encoding:
import tensorflow as tf

# Preprocess the data
train_set_x = train_set_x.astype('float32') / 255
valid_set_x= valid_set_x.astype('float32') / 255
test_set_x = test_set_x.astype('float32') / 255

test_set_y = tf.keras.utils.to_categorical(test_set_y, num_classes=2)
valid_set_y = tf.keras.utils.to_categorical(valid_set_y, num_classes=2)
test_set_y = tf.keras.utils.to_categorical(test_set_y, num_classes=2)


2023-04-04 01:36:52.143661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 01:36:52.804785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-04 01:36:52.804813: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-04 01:36:54.406588: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(250, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_set_x, train_set_y, batch_size=32, epochs=50, validation_data=(valid_set_x, valid_set_y))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_set_x, test_set_y)
print("Test accuracy:", test_acc)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping

# # Preprocess the data
# x_train = train_set_x.astype('float32') / 255
# x_val = valid_set_x.astype('float32') / 255
# x_test = test_set_x.astype('float32') / 255

# # Define the CNN model architecture
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(100, 250, 1)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model with early stopping based on validation loss
# early_stop = EarlyStopping(monitor='val_loss', patience=3)
# model.fit(x_train, train_set_y, epochs=20, batch_size=32, validation_data=(valid_set_x, valid_set_y),
#           callbacks=[early_stop])

# # Evaluate the model on the testing set
# test_loss, test_acc = model.evaluate(x_test, y_test)
# print(f'Testing loss: {test_loss}, Testing accuracy: {test_acc}')

#### Step 2 - Train an autoencoder to allow a sequential model to operate

In [ ]:
# PART 2:
    
# with h5py.File('combustion_img_13.mat', 'r') as f:
#     train_set_x = np.array(f['train_set_x'])
#     valid_set_x = np.array(f['valid_set_x'])
#     test_set_x = np.array(f['test_set_x'])
    
# x_train = train_set_x.astype('float32') / 255.
# x_val = valid_set_x.astype('float32') / 255.
# x_test = test_set_x.astype('float32') / 255.

# x_train = x_train.reshape(-1, 250, 100, 1)
# x_val = x_val.reshape(-1, 250, 100, 1)
# x_test = x_test.reshape(-1, 250, 100, 1)


# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D

# input_img = Input(shape=(250, 100, 1))

# x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
# x = MaxPooling2D((2, 2), padding='same')(x)
# x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# encoded = MaxPooling2D((2, 2), padding='same')(x)

# x = Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same')(encoded)
# x = Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
# x = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
# decoded = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# autoencoder = Model(input_img, decoded)
# autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# TEST CODE:

In [ ]:
# import numpy as np
# import scipy.io as sio    
# import matplotlib.pyplot as plt

# # Load the data
# # data = sio.loadmat('combustion_img_13.mat')  #NOTE: Found out scipy.io doesn't support Matlab v7.3 data files

# import h5py
# with h5py.File('combustion_img_13.mat', 'r') as f:

# # print(list(f.keys())) #Print the available keys in the file
    
# # Get the training, validation, and testing sets
#     train_set_x = np.array(f['train_set_x'])
#     train_set_y = np.array(f['train_set_y'])
#     valid_set_x = np.array(f['valid_set_x'])
#     valid_set_y = np.array(f['valid_set_y'])
#     test_set_x = np.array(f['test_set_x'])
#     test_set_y = np.array(f['test_set_y'])  

# # Define the classes
# classes = ['stable', 'unstable']

# # Visualize a few images
# # fig, axs = plt.subplots(2, 5, figsize=(10, 5))
# # axs = axs.ravel()
# # for i in range(10):
# #     axs[i].imshow(train_set_x[i], cmap='gray')
# #     axs[i].set_title(classes[train_set_y[i][0]])
# #     axs[i].axis('off')
# plt.imshow(train_set_x[0], cmap = 'gray')
# plt.show()
